## Tax calculator

In [ ]:
#Employee class
class Employee:
    #constructor of employee class
    def __init__(self, StaffID, LastName, FirstName, RegHours, HourlyRate, OTMultiple, TaxCredit, StandardBand):
        self.StaffID = StaffID
        self.LastName = LastName
        self.FirstName = FirstName
        self.RegHours = RegHours
        self.HourlyRate = HourlyRate
        self.OTMultiple = OTMultiple
        self.TaxCredit = TaxCredit
        self.StandardBand = StandardBand
        
    #Compute payment takes hours worked and date and returns the json
    def computePayment(self, HoursWorked, date):
        o = {}
        o['Hours Worked'] = HoursWorked
        o['Regular Hours Worked'] = self.RegHours
        o['Regular Rate'] = self.HourlyRate
        OTMultiple = self.OTMultiple
        o['Tax Credit'] = self.TaxCredit
        StandardBand = self.StandardBand
        o['Date'] = date
        o['Name'] = self.FirstName + ' ' + self.LastName
        o['Regular Pay'] = o['Regular Rate'] * o['Regular Hours Worked']
        # Regular Hours Worked cannot exceed hours worked
        if (o['Hours Worked'] > o['Regular Hours Worked']):
            o['Overtime Hours Worked'] = o['Hours Worked'] - o['Regular Hours Worked']
        else:
            o['Overtime Hours Worked'] = 0
        o['OverTime Rate'] = int(o['Regular Rate'] * OTMultiple)
        # Overtime pay or overtime hours cannot be negative.
        o['OverTime Pay'] = o['Overtime Hours Worked'] * o['OverTime Rate']
        o['Gross Pay'] = o['Regular Pay'] + o['OverTime Pay']
        o['Standard Rate Pay'] = StandardBand
        if (o['Gross Pay'] > o['Standard Rate Pay']):
            o['Higher Rate Pay'] = o['Gross Pay'] - o['Standard Rate Pay']
        else:
            o['Higher Rate Pay'] = 0
        o['Standard Tax'] = int(o['Standard Rate Pay'] * 0.2)
        # Higher Tax cannot be negative.
        o['Higher Tax'] = round(o['Higher Rate Pay'] * 0.4, 2)
        o['Total Tax'] = round(o['Standard Tax'] + o['Higher Tax'], 2)
        o['Net Deduction'] = round(o['Total Tax'] - o['Tax Credit'], 2)
        # Net Pay cannot be negative.
        o['Net Pay'] = round(o['Gross Pay'] - o['Net Deduction'], 2)
        return o
    
    #static method to perform bulk operation
    @staticmethod
    def bulk_tax_compute(employeeFile, hourFile):
        employdata = {}
        result = []
        try:
            #read employee file
            with open(employeeFile) as newFile:
                for line in newFile:
                    StaffID, LastName, FirstName, RegHours, HourlyRate, OTMultiple, TaxCredit, StandardBand = line.split()
                    employdata[StaffID] = Employee(StaffID, LastName, FirstName, int(RegHours), int(HourlyRate),
                                                   float(OTMultiple),
                                                   int(TaxCredit), int(StandardBand))
            
        except Exception as ex:
            print(ex)
            raise Exception("Invalid Input file")
        return result